In [1]:
import re
import pandas as pd
import csv
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from http_request_randomizer.requests.proxy.requestProxy import RequestProxy

In [2]:
# 1. Get zip codes
csv_file = r'C:\Users\Beatrice Tierra\Documents\Springboard\US-Hospital-Charges\Datasets\Addresses.csv'
address_df = pd.read_csv(csv_file)
zipcodes = [re.search('CA \d\d\d\d\d', add).group(0).split(' ')[1] for add in address_df['Address']]

In [3]:
print(zipcodes)

['95901', '93301', '95422', '91206', '95490', '95240', '93654', '93065', '95370', '94574', '95482', '94590', '90033', '93230', '92801', '94501', '91801', '94705', '94609', '92120', '91942', '93534', '92324', '95401', '91724', '91107', '92128', '93001', '93309', '93308', '93301', '92411', '96130', '90026', '92311', '96150', '92315', '90640', '91770', '90015', '94114', '94110', '91710', '91768', '90704', '90048', '90301', '94618', '94158', '90027', '92868', '94133', '91710', '91010', '93611', '90650', '92627', '90255', '92411', '93940', '93003', '93721', '94553', '92882', '95203', '90505', '93215', '92262', '92395', '95336', '95350', '95065', '90806', '90023', '96122', '94546', '92270', '94040', '92243', '91741', '91790', '95382', '91436', '93309', '95926', '96097', '92780', '92708', '93401', '93710', '91754', '93930', '91204', '91740', '93111', '93308', '90017', '95124', '91733', '95023', '95448', '92866', '92252', '94602', '92663', '92618', '91105', '94112', '95338', '92201', '94520', 

In [4]:
# 2. Get proxy list
req_proxy = RequestProxy() #you may get different number of proxy when  you run this at each time
proxies = req_proxy.get_proxy_list() #this will create proxy list

2021-05-17 17:23:20,654 http_request_randomizer.requests.useragent.userAgent INFO     Using local file for user agents: C:\Users\Beatrice Tierra\anaconda3\lib\site-packages\http_request_randomizer\requests\proxy\../data/user_agents.txt
2021-05-17 17:23:20,656 root   DEBUG    === Initialized Proxy Parsers ===
2021-05-17 17:23:20,657 root   DEBUG    	 FreeProxy parser of 'http://free-proxy-list.net' with required bandwidth: '150' KBs
2021-05-17 17:23:20,658 root   DEBUG    	 PremProxy parser of 'https://premproxy.com/list/' with required bandwidth: '150' KBs
2021-05-17 17:23:20,659 root   DEBUG    	 SslProxy parser of 'https://www.sslproxies.org' with required bandwidth: '150' KBs
2021-05-17 17:23:20,660 root   DEBUG    =================================
2021-05-17 17:23:25,505 root   DEBUG    Added 300 proxies from FreeProxy
2021-05-17 17:23:31,189 http_request_randomizer.requests.parsers.PremProxyParser WARNING  Proxy Provider url failed: https://premproxy.com/list/
2021-05-17 17:23:31,

In [8]:
# 3. Get cost after insurance 
codes = ['99282', '99283', '99284', '99285']
with open('CoverageCost.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['CPT Code', 'Zip', 'In-Network', 'Out-Network'])
    for i, code in enumerate(codes):
        PROXY = proxies[i].get_address()
        for zip in zipcodes:
            webdriver.DesiredCapabilities.CHROME['proxy']={"httpProxy":PROXY,"ftpProxy":PROXY,"sslProxy":PROXY,"proxyType":"MANUAL",}
            webdriver.DesiredCapabilities.CHROME['acceptSslCerts']=True
            d = webdriver.Chrome(r'C:\Users\Beatrice Tierra\Downloads\chromedriver.exe')
            time.sleep(10)
            d.get('https://www.fairhealthconsumer.org/medical/zip')
            
            time.sleep(10)
            location_search = d.find_element_by_css_selector("input[placeholder='Zip Code or City, State (e.g, 12345 or New York, NY)']")
            location_search.send_keys(zip)
            location_search.send_keys(Keys.ENTER)

            time.sleep(10)
            d.find_element_by_xpath("//a[@href='/medical/select-medical-totalcost']").click()

            time.sleep(10)
            cpt_search = d.find_element_by_css_selector("input[placeholder='Enter a CPT Code or Keyword']")
            cpt_search.send_keys('99282')
            cpt_search.send_keys(Keys.ENTER)

            time.sleep(10)
            d.find_element_by_css_selector('[class="button agree"]').click()

            time.sleep(10)
            button = d.find_element_by_css_selector("input[type='radio'][value='99282']")
            d.execute_script("arguments[0].click();", button)

            time.sleep(10)
            d.find_element_by_css_selector('[class="inline-cost-div arrow-btn"]').click()

            time.sleep(10)
            out_net = d.find_element_by_css_selector('[class="circle out-net-summary"]').text
            in_net = d.find_element_by_css_selector('[class="circle in-net-summary"]').text

            writer.writerow([code, zip, in_net, out_net])
            d.close()

2021-05-17 17:29:25,677 selenium.webdriver.remote.remote_connection DEBUG    POST http://127.0.0.1:63435/session {"capabilities": {"firstMatch": [{}], "alwaysMatch": {"browserName": "chrome", "platformName": "any", "proxy": {"httpProxy": "151.232.72.21:80", "ftpProxy": "151.232.72.21:80", "sslProxy": "151.232.72.21:80", "proxyType": "manual"}, "acceptInsecureCerts": true, "goog:chromeOptions": {"extensions": [], "args": []}}}, "desiredCapabilities": {"browserName": "chrome", "version": "", "platform": "ANY", "proxy": {"httpProxy": "151.232.72.21:80", "ftpProxy": "151.232.72.21:80", "sslProxy": "151.232.72.21:80", "proxyType": "MANUAL"}, "acceptSslCerts": true, "goog:chromeOptions": {"extensions": [], "args": []}}}
2021-05-17 17:29:26,261 selenium.webdriver.remote.remote_connection DEBUG    Finished Request
2021-05-17 17:29:36,265 selenium.webdriver.remote.remote_connection DEBUG    POST http://127.0.0.1:63435/session/de297e162b23cf4e6f9e85eed050d858/url {"url": "https://www.fairhealthc

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"input[placeholder='Zip Code or City, State (e.g, 12345 or New York, NY)']"}
  (Session info: chrome=90.0.4430.212)
